# 長期記憶（Long-term Memory）
在前面的章節中，我們使用 MemorySaver 來保存單一對話的狀態。但實際應用中，我們常需要：
- 記住使用者的偏好（跨對話）
- 儲存重要的事實和知識
- 在不同對話間共享資訊

LangGraph 的 Store 介面提供了長期記憶功能，讓 AI 可以跨 thread 記住資訊。
## Store 介面概述
Store 是一個鍵值儲存系統，與 Checkpointer 的差異：

| 特性 | Checkpointer | Store |
|------|--------------|-------|
| 用途 | 保存對話狀態 | 儲存長期記憶 |
| 範圍 | 單一 thread | 跨所有 thread |
| 生命週期 | 隨對話結束 | 永久保存 |
| 典型用途 | 對話歷史 | 使用者偏好、知識庫 |


## InMemoryStore（記憶體儲存）
LangGraph 提供 InMemoryStore 作為最簡單的 Store 實作。

In [ ]:
from langgraph.store.memory import InMemoryStore
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict

# 建立 Store
store = InMemoryStore()

class ChatState(TypedDict):
    messages: list[dict]
    user_id: str

def chat_node(state: ChatState, store):
    """聊天節點，可以存取 store"""
    user_id = state["user_id"]
    
    # 從 store 讀取使用者名稱
    user_data = store.get(("users", user_id), "profile")
    user_name = user_data.value.get("name", "訪客") if user_data else "訪客"
    
    user_msg = state["messages"][-1]["content"]
    response = f"你好 {user_name}！你說：{user_msg}"
    
    return {
        "messages": state["messages"] + [
            {"role": "assistant", "content": response}
        ]
    }

# 建立 graph
workflow = StateGraph(ChatState)
workflow.add_node("chat", chat_node)
workflow.set_entry_point("chat")
workflow.add_edge("chat", END)

checkpointer = MemorySaver()
graph = workflow.compile(checkpointer=checkpointer, store=store)

# 先儲存使用者資料到 store
store.put(("users", "user_123"), "profile", {"name": "小明", "age": 25})

# 執行對話
config = {"configurable": {"thread_id": "chat_001"}}
result = graph.invoke({
    "messages": [{"role": "user", "content": "你好"}],
    "user_id": "user_123"
}, config)

print(result["messages"][-1]["content"])

你好 小明！你說：你好


## Store 的基本操作：


In [ ]:
# 儲存資料
store.put(
    namespace=("users", "user_123"),  # 命名空間
    key="profile",                     # 鍵
    value={"name": "小明", "age": 25} # 值
)

# 讀取資料
item = store.get(("users", "user_123"), "profile")
if item:
    data = item.value # 透過 .value 取得實際資料
    print(data)  # {'name': '小明', 'age': 25}

# 搜尋資料
results = store.search(("users",))  # 回傳 Item 物件的列表
for item in results:
    print(item.value)  # 存取每個 Item 的值

# 刪除資料
store.delete(("users", "user_123"), "profile")

### 簡易範例：個人知識庫

In [6]:
from langgraph.store.memory import InMemoryStore
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict, Annotated
from operator import add
from datetime import datetime

store = InMemoryStore()

class KnowledgeState(TypedDict):
    messages: Annotated[list[dict], add]
    user_id: str

def knowledge_node(state: KnowledgeState, store):
    """處理知識儲存和查詢"""
    user_id = state["user_id"]
    user_msg = state["messages"][-1]["content"]
    
    # 儲存事實
    if user_msg.startswith("記住："):
        fact = user_msg.replace("記住：", "").strip()
        
        # 儲存到知識庫
        timestamp = datetime.now().isoformat()
        fact_id = f"fact_{timestamp}"
        
        store.put(
            ("knowledge", user_id),
            fact_id,
            {
                "content": fact,
                "timestamp": timestamp,
                "category": "user_fact"
            }
        )
        
        response = f"✓ 已記住：{fact}"
    
    # 查詢知識
    elif "記得什麼" in user_msg or "告訴我" in user_msg:
        # 搜尋所有知識
        facts = store.search(("knowledge", user_id))
        
        if facts:
            fact_list = [f"• {item.value['content']}" for item in facts]
            response = "我記得這些：\n" + "\n".join(fact_list)
        else:
            response = "我還沒有記住任何關於你的資訊"
    
    else:
        response = "你可以說「記住：...」讓我記住資訊，或問「你記得什麼」"
    
    return {
        "messages": [{"role": "assistant", "content": response}]
    }

workflow = StateGraph(KnowledgeState)
workflow.add_node("knowledge", knowledge_node)
workflow.set_entry_point("knowledge")
workflow.add_edge("knowledge", END)

graph = workflow.compile(checkpointer=MemorySaver(), store=store)

user_id = "user_456"

# 儲存多個事實
facts = [
    "記住：我的生日是 5 月 15 日",
    "記住：我住在台北",
    "記住：我的寵物叫小白"
]

for i, fact in enumerate(facts):
    config = {"configurable": {"thread_id": f"thread_{i}"}}
    result = graph.invoke({
        "messages": [{"role": "user", "content": fact}],
        "user_id": user_id
    }, config)
    print(result["messages"][-1]["content"])

print("\n--- 查詢知識 ---")
# 查詢所有事實
config = {"configurable": {"thread_id": "query_thread"}}
result = graph.invoke({
    "messages": [{"role": "user", "content": "你記得什麼"}],
    "user_id": user_id
}, config)
print(result["messages"][-1]["content"])

✓ 已記住：我的生日是 5 月 15 日
✓ 已記住：我住在台北
✓ 已記住：我的寵物叫小白

--- 查詢知識 ---
我記得這些：
• 我的生日是 5 月 15 日
• 我住在台北
• 我的寵物叫小白


## Namespace 管理

Namespace 用來組織和隔離不同類型的記憶，類似資料夾結構。

In [7]:
from langgraph.store.memory import InMemoryStore

store = InMemoryStore()

# 使用 tuple 定義階層式 namespace
# 格式：(類型, 子類型, ID)

# 1. 使用者相關資料
store.put(("users", "user_123"), "profile", {
    "name": "小明",
    "email": "ming@example.com"
})

store.put(("users", "user_123"), "settings", {
    "theme": "dark",
    "language": "zh-TW"
})

# 2. 偏好設定
store.put(("preferences", "user_123"), "food", {
    "likes": ["咖啡", "壽司"],
    "dislikes": ["香菜"]
})

store.put(("preferences", "user_123"), "hobbies", {
    "items": ["閱讀", "旅遊", "攝影"]
})

# 3. 對話歷史摘要
store.put(("summaries", "user_123"), "2025-12", {
    "topics": ["AI", "旅遊規劃"],
    "interactions": 15
})

# 4. 知識庫
store.put(("knowledge", "user_123"), "fact_001", {
    "content": "喜歡喝黑咖啡",
    "category": "preference"
})

# 搜尋特定 namespace（回傳 Item 列表）
user_data = store.search(("users", "user_123"))
print("使用者資料:")
for item in user_data:
    print(f"  {item.key}: {item.value}")

preferences = store.search(("preferences", "user_123"))
print("\n偏好設定:")
for item in preferences:
    print(f"  {item.key}: {item.value}")

# 搜尋所有使用者的偏好
all_preferences = store.search(("preferences",))
print("\n所有偏好:")
for item in all_preferences:
    print(f"  {item.namespace}: {item.value}")

使用者資料:
  profile: {'name': '小明', 'email': 'ming@example.com'}
  settings: {'theme': 'dark', 'language': 'zh-TW'}

偏好設定:
  food: {'likes': ['咖啡', '壽司'], 'dislikes': ['香菜']}
  hobbies: {'items': ['閱讀', '旅遊', '攝影']}

所有偏好:
  ('preferences', 'user_123'): {'likes': ['咖啡', '壽司'], 'dislikes': ['香菜']}
  ('preferences', 'user_123'): {'items': ['閱讀', '旅遊', '攝影']}


## Namespace 設計最佳實踐：

In [ ]:
# 好的設計：清晰的階層
("users", user_id)                      # 使用者資料
("preferences", user_id)                # 使用者偏好
("knowledge", user_id)                  # 使用者知識
("sessions", user_id)                   # 對話會話
("analytics", user_id)                  # 分析資料

# 不好的設計：扁平化
("user_profile",)                       # 無法區分使用者
("user_123_food_preference",)           # 難以搜尋和管理

## 完整範例：個人助理系統

結合所有長期記憶功能：

In [1]:
from langgraph.store.memory import InMemoryStore
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict, Annotated
from operator import add
from datetime import datetime

store = InMemoryStore()

class AssistantState(TypedDict):
    messages: Annotated[list[dict], add]
    user_id: str

def assistant_node(state: AssistantState, store):
    user_id = state["user_id"]
    user_msg = state["messages"][-1]["content"].lower()
    
    # 初始化使用者資料
    profile_item = store.get(("users", user_id), "profile")
    if not profile_item:
        store.put(("users", user_id), "profile", {
            "name": "新使用者",
            "created_at": datetime.now().isoformat()
        })
        profile = {"name": "新使用者"}
    else:
        profile = profile_item.value
    
    user_name = profile.get("name", "朋友")
    
    # 命令處理
    if "叫我" in user_msg:
        # 更新名稱
        new_name = user_msg.split("叫我")[-1].strip()
        store.put(("users", user_id), "profile", {
            **profile,
            "name": new_name
        })
        response = f"好的！我會叫你 {new_name}"
    
    elif "我喜歡" in user_msg:
        # 儲存偏好
        item = user_msg.split("我喜歡")[-1].strip()
        prefs_item = store.get(("preferences", user_id), "likes")
        likes = prefs_item.value.get("items", []) if prefs_item else []
        likes.append(item)
        
        store.put(("preferences", user_id), "likes", {"items": likes})
        response = f"記住了！{user_name} 喜歡 {item}"
    
    elif "記住" in user_msg:
        # 儲存知識
        fact = user_msg.split("記住")[-1].strip()
        fact_id = f"fact_{datetime.now().timestamp()}"
        
        store.put(("knowledge", user_id), fact_id, {
            "content": fact,
            "timestamp": datetime.now().isoformat()
        })
        response = f"已記住：{fact}"
    
    elif "你知道我" in user_msg or "關於我" in user_msg:
        # 彙整所有記憶
        prefs_item = store.get(("preferences", user_id), "likes")
        likes = prefs_item.value.get("items", []) if prefs_item else []
        
        facts = list(store.search(("knowledge", user_id)))
        fact_contents = [f.value["content"] for f in facts]
        
        info = [f"你的名字是 {user_name}"]
        if likes:
            info.append(f"你喜歡：{', '.join(likes)}")
        if fact_contents:
            info.append(f"關於你：{'; '.join(fact_contents)}")
        
        response = "\n".join(info)
    
    else:
        response = f"你好 {user_name}！我可以記住你的偏好和資訊。試試說：\n• 叫我 [名字]\n• 我喜歡 [東西]\n• 記住 [事實]\n• 你知道我什麼"
    
    return {
        "messages": [{"role": "assistant", "content": response}]
    }

workflow = StateGraph(AssistantState)
workflow.add_node("assistant", assistant_node)
workflow.set_entry_point("assistant")
workflow.add_edge("assistant", END)

graph = workflow.compile(checkpointer=MemorySaver(), store=store)

# 模擬對話
user_id = "user_789"

conversations = [
    "你好",
    "叫我小華",
    "我喜歡爬山",
    "我喜歡攝影",
    "記住我每週日去爬山",
    "你知道我什麼？"
]

for i, msg in enumerate(conversations):
    config = {"configurable": {"thread_id": f"thread_{i}"}}
    result = graph.invoke({
        "messages": [{"role": "user", "content": msg}],
        "user_id": user_id
    }, config)
    
    print(f"\n使用者: {msg}")
    print(f"助理: {result['messages'][-1]['content']}")


使用者: 你好
助理: 你好 新使用者！我可以記住你的偏好和資訊。試試說：
• 叫我 [名字]
• 我喜歡 [東西]
• 記住 [事實]
• 你知道我什麼

使用者: 叫我小華
助理: 好的！我會叫你 小華

使用者: 我喜歡爬山
助理: 記住了！小華 喜歡 爬山

使用者: 我喜歡攝影
助理: 記住了！小華 喜歡 攝影

使用者: 記住我每週日去爬山
助理: 已記住：我每週日去爬山

使用者: 你知道我什麼？
助理: 你的名字是 小華
你喜歡：爬山, 攝影
關於你：我每週日去爬山


# 語意搜尋與檢索

在前面的章節中，我們使用 Store 來儲存和檢索結構化資料。但當資料量增大時，我們需要更智慧的搜尋方式——語意搜尋。它能理解查詢的「意義」，而不只是關鍵字配對。

### to be continued